In [1]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [2]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
from os import environ
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [3]:
root = "/data/gpfs/assoc/denovo/tfaske/sagebrush/"

In [4]:
cd $root

/data/gpfs/assoc/denovo/tfaske/sagebrush


In [5]:
pwd

'/data/gpfs/assoc/denovo/tfaske/sagebrush'

# Actual Mapping 
 

In [6]:
fastq_files = []
for seqdir in ['fastq/']:
    os.chdir('{}/{}'.format(root, seqdir))
    files = !find . -name '*fq.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [7]:
len(fastq_files),fastq_files[0]

(708, '/data/gpfs/assoc/denovo/tfaske/sagebrush/fastq/AT2_BB_1.F.fq.gz')

In [8]:
denovo_dir = os.path.join(root,"denovo")
assert(denovo_dir)

In [9]:
cd $denovo_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo


In [10]:
assembly = "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta"

#assembly indexed with dDocent 

In [11]:
# -k INT minimum seed length [19]
# -w INT band width for banded alignment [100]
# -r FLOAT look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
# -T INT minimum score to output [30]
# -R STR read group header line such as '@RG\tID:foo\tSM:bar' [null]

#@lview.remote()
def run_bwamem(args):
    import os, multiprocessing, socket
    cpus = 1
    assembly, fq, outdir = args
    ID = fq.split('/')[8] ### need to change this to match your ID 
    ID = ID.split('.F.')[0] ### This too 
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(ID)))
    bam = sam.replace('.sam','.bam')
    bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
    bwa_cmd = r"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\tID:%s\tLB:%s\tSM:%s\tPL:ILLUMINA' %s %s > %s" % (ID,ID,ID,assembly,fq,sam)
    s2b_cmd =  "samtools view -b %s -o %s\n\nsamtools sort -@ %s %s -o %s\n\nsamtools index %s\n\n" % (sam,bam,cpus,bam,bam_sorted,bam_sorted)                                                              
    return  bwa_cmd,s2b_cmd 

In [12]:
!mkdir SNPcall

mkdir: cannot create directory ‘SNPcall’: File exists


In [13]:
!mkdir SNPcall/bwa

mkdir: cannot create directory ‘SNPcall/bwa’: File exists


In [14]:
!mkdir SNPcall/shbwa

In [15]:
bwa_dir = os.path.join(denovo_dir,"SNPcall/bwa/")
assert(bwa_dir)

In [16]:
### creates a list of commands for bwa-mem for each fastq file
res_bwa = []
res_s2b = []
for f in fastq_files:
    r1,r2 = run_bwamem((assembly, f, bwa_dir))
    res_bwa.append(r1)
    res_s2b.append(r2)

In [17]:
len(res_bwa),res_bwa[0]

(708,
 "bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:AT2_BB_1\\tLB:AT2_BB_1\\tSM:AT2_BB_1\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/sagebrush/fastq/AT2_BB_1.F.fq.gz > /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1.sam")

In [18]:
len(res_s2b),res_s2b[0]

(708,
 'samtools view -b /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1.sam -o /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1.bam\n\nsamtools sort -@ 1 /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1.bam -o /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1_sorted.bam\n\nsamtools index /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_1_sorted.bam\n\n')

In [19]:
pwd

'/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo'

#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [23]:
fq_ID = [fq.split('/')[8].split('.F.')[0] for fq in fastq_files]

In [55]:
### select options for slurm submission
#account = 'cpu-s5-denovo-0'
#partition = 'cpu-core-0'
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'bwa_ARTR'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [56]:
def write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,bwa_cmds,s2b_cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(bwa_cmds)):
        with open("SNPcall/shbwa/run_bwamem_%s.sh" % (fq_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output output_bwamem_%s.txt
#SBATCH --mail-user=%s \n\n
    
%s \n\n
%s \n""" % (account,partition,time,int(cpus),int(mem_cpu),fq_ID[i],fq_ID[i],email,bwa_cmds[i],s2b_cmds[i]))

In [57]:
write_bwamem_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,fq_ID,res_bwa,res_s2b)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [58]:
shbwa_files = []
os.chdir('{}/{}'.format(denovo_dir, 'SNPcall/shbwa'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [59]:
len(shbwa_files),shbwa_files[0]

(708,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/shbwa/run_bwamem_AT2_BB_1.sh')

In [60]:
def write_bash_bwamem_sh(sh_files):
    with open("SNPcall/run_bash_bwa.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [61]:
cd $denovo_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo


In [62]:
write_bash_bwamem_sh(shbwa_files)

# Run run_bash_bwamem_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall 
    source activate py36
    bash run_bash_bwa.sh
    
## calculate bam_coverages and move good files

        already done with OG analysis. Just use 

In [63]:
pwd

'/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo'

In [64]:
cov_df = pd.read_csv('SNPcall/bam_coverage.csv')

In [65]:
cov_df.coverage.describe()


count    708.000000
mean      35.644292
std       11.286788
min        1.696807
25%       28.291483
50%       34.349371
75%       43.191507
max       69.359485
Name: coverage, dtype: float64

In [66]:
len(cov_df.coverage)

708

# move good bams

In [67]:
len(cov_df[cov_df.coverage > 10]),len(cov_df[cov_df.coverage >= 5])

(695, 698)

In [68]:
good_bam = cov_df.bam[cov_df.coverage >= 10]
good_bam = good_bam.tolist()
len(good_bam),good_bam[0]

(695, 'AT2_BB_10_sorted.bam')

In [70]:
good_bam_files = []
for i in range(0,len(good_bam)):
    bam = bwa_dir + good_bam[i]
    good_bam_files.append(bam)
len(good_bam_files), good_bam_files[1]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_11_sorted.bam')

In [74]:
snp_dir = os.path.join(denovo_dir,'SNPcall')

In [75]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall


In [76]:
!mkdir 'good_bams'

mkdir: cannot create directory ‘good_bams’: File exists


In [77]:
good_bam_dir = os.path.join(snp_dir,'good_bams')
assert good_bam_dir

In [78]:
for i in range(0,len(good_bam_files)):
    good_bam = str(good_bam_files[i])
    !cp $good_bam $good_bam_dir

cp: cannot stat ‘/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AV2_CF_1_sorted.bam’: No such file or directory
cp: cannot stat ‘/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AV2_CF_2_sorted.bam’: No such file or directory
cp: cannot stat ‘/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AV2_CF_3_sorted.bam’: No such file or directory


#### AV2 changed to AV4

In [80]:
cd $good_bam_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/good_bams


In [81]:
good_bam_cp_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    good_bam_cp_files.append(x)
good_bam_cp_files = sorted(good_bam_files)

In [82]:
len(good_bam_cp_files),good_bam_cp_files[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/bwa/AT2_BB_10_sorted.bam')